In [ ]:
import pandas as pd
import numpy as np
import dataframe_image as dfi

import pickle

In [ ]:
# 데이터프레임을 그림 파일로 저장
option = 3 # 1 : dfi / 2 : df2img / 3 : do nothing
import df2img
def savedf(df, path):
    if option == 1:
        dfi.export(df, path)
    if option == 2:
        df2img.save_dataframe(fig=df2img.plot_dataframe(df), filename=path)
    if option == 3:
        pass

# 경기도 42개 지역(시, 군, 구) 선정

In [ ]:
# 행정구역상 경기도에 소속되어 있는 지역들 중
# 구(區)가 존재하지 않는 시(市)의 경우에는 시를
# 구가 존재하는 시는 구를 고려하면, 총 42개의 지역을 선정할 수 있다.
# 출처 : https://www.gg.go.kr/contents/contents.do?ciIdx=468&menuId=1831
regions = pd.read_csv("../../data/data_b_raw/2b200_1230_rows.csv")

In [ ]:
# 지역목록 1 ; "xx시", "xx군", "xx시 xx구"
regions = sorted(list(regions['지역']))
print(regions)

In [ ]:
# 지역목록 2 ; "xx시", "xx군", "xx구"
regions_shortened = [i.split() for i in regions] # list of lists
regions_shortened = sum(regions_shortened, []) # flattening
regions_shortened = [region for region in regions_shortened if region not in ['고양시', '성남시', '수원시', '안산시', '안양시', '용인시']]
print(regions_shortened)

In [ ]:
shorten = dict(zip(regions, regions_shortened)) # "xx시 xx구"를 "xx구"로 바꾼다.
lengthen = dict(zip(regions_shortened, regions)) # "xx구"를 "xx시 xx구"로 바꾼다.
print(shorten['수원시 팔달구'])
print(lengthen['팔달구'])

In [ ]:
np.array(regions)[np.array([len(region.split()) for region in regions])==1]
np.array(regions)[np.array([len(region.split()) for region in regions])==2]


In [ ]:
# 시 목록 # 25개
city_list = np.array(regions)[np.array([len(region.split()) for region in regions])==1]
# 구 목록 1 # 17개
district_list = np.array(regions)[np.array([len(region.split()) for region in regions])==2]
# # 구 목록 2 # 17개
district_list_ = [shorten[region] for region in district_list]
print("시 목록("+str(len(city_list))+"개) : ")
print(city_list)
print("구 목록("+str(len(district_list))+"개) : ")
print(district_list)
print(district_list_)

## 지역별 위도, 경도

In [ ]:
locations = pd.read_csv("../../data/data_b_raw/2b205_0110_locations.csv")
locations.set_index("지역", inplace=True)
locations.rename(columns={'latitude':'위도', 'longitude':'경도'}, inplace=True)
locations.head()

# 각 feature별 데이터 수집

## 쏘카존 수 : `zcount`

In [ ]:
remove_u3000 = { 
    '수원시\u3000팔달구':'수원시 팔달구', '수원시\u3000영통구':'수원시 영통구', '용인시\u3000기흥구':'용인시 기흥구',
    '고양시\u3000일산동구':'고양시 일산동구', '안양시\u3000동안구':'안양시 동안구', '고양시\u3000일산서구':'고양시 일산서구',
    '성남시\u3000수정구':'성남시 수정구', '수원시\u3000장안구':'수원시 장안구', '성남시\u3000중원구':'성남시 중원구',
    '안산시\u3000단원구':'안산시 단원구', '용인시\u3000수지구':'용인시 수지구', '고양시\u3000덕양구':'고양시 덕양구',
    '용인시\u3000처인구':'용인시 처인구', '안양시\u3000만안구':'안양시 만안구', '성남시\u3000분당구':'성남시 분당구',
    '수원시\u3000권선구':'수원시 권선구', '안산시\u3000상록구':'안산시 상록구'}
with open('results/1b203_0208_remove_u3000.pkl', 'wb') as f:
    pickle.dump(remove_u3000, f)


In [ ]:
# 쏘카존 갯수 데이터(김선중)
# 출처 : (aiffel, going deeper 4-6)
zones = pd.read_csv("../../data/data_b_raw/2b201_1228_20210823_socar_zone.csv")
zones = zones[zones['region1']=="경기도"] # 경기도 자료만 자르기
zones = zones.reset_index(drop=True)
zones['region2'].replace(remove_u3000, inplace=True)
zones.head()

In [ ]:
# 각 지역별 쏘카존 수 데이터 생성
zcount = pd.DataFrame(zones['region2'].value_counts())
zcount.columns = ['쏘카존 수']
zcount = zcount.reindex(regions)
zcount.head()

## 토지용도 : `land_use_`

In [ ]:
# 동, 면별 토지용도 데이터 1(윤겸님)
# 데이터를 얻은 곳 : http://gris.gg.go.kr/ost/oneStopView.do?pnu=4150010300104450002
land_use1 = pd.read_csv("../../data/data_b_raw/2b202_1228_토지이용계획_차윤겸_최종.csv", encoding='CP949')
land_use2 = pd.read_csv("../../data/data_b_raw/2b203_1228_토지이용계획_김선중_최종.csv",encoding='CP949')
mask = land_use1['시, 구'].isin(list(set(land_use1['시, 구']).intersection(land_use2['시, 구'])))
land_use1 = land_use1[~mask]
land_use = pd.concat([land_use1,land_use2])
land_use.reset_index(inplace=True, drop=True)
land_use

In [ ]:
land_use_ = pd.DataFrame(index = regions)
nature_list = []
dwell_list = []
industry_list = []
commerce_list = []
for region in regions:
    ratio_region = land_use[land_use['시, 구']==region]
    total_number = len(ratio_region)
    nature_list.append(sum(ratio_region['토지용도']=='녹지')/total_number)
    dwell_list.append(sum(ratio_region['토지용도']=='주거')/total_number)
    industry_list.append(sum(ratio_region['토지용도']=='공업')/total_number)
    commerce_list.append(sum(ratio_region['토지용도']=='상업')/total_number)
# 각 지역별 녹지/주거/공업/상업의 동/면 비율 데이터 생성
land_use_['녹지'] = nature_list
land_use_['주거'] = dwell_list
land_use_['공업'] = industry_list
land_use_['상업'] = commerce_list
land_use_.head()

## 인구 : `pop`

In [ ]:
# 경기지역 행정구역별 인구 데이터(연수님)
# 데이터를 얻은 곳 : [출처필요]
pop = pd.read_csv("../../data/data_b_raw/2b301_1227_행정구역_시군구_별__성별_인구수.csv", encoding='cp949')
pop.head()

In [ ]:
pop.set_index('행정구역(시군구)별',inplace=True)
pop.drop(['행정구역(시군구)별', '경기도', '수원시', '성남시', '안양시', '송탄출장소', '안중출장소', '안산시', '고양시',
           '풍양출장소', '용인시', '화성시동부출장소', '화성시동탄출장소'], inplace=True)
pop = pd.DataFrame(pop['2022.11'])
pop.reset_index(inplace=True)
pop['행정구역(시군구)별'].replace(lengthen, inplace=True) # "xx구"를 "xx시 xx구"로 바꾼다.
pop.rename(columns={'2022.11':'인구'},inplace=True)
pop.set_index('행정구역(시군구)별',inplace=True)
pop = pop.reindex(regions)
pop.head()

In [ ]:
sorted(pop.index.unique()) == regions # 앞서 선정한 42개 지역 목록과 일치한다.

## 대학교 수 : `univ_`

In [ ]:
# 경기지역 대학교 목록(연수님)
# 데이터를 얻은 곳 : [출처필요]
univ = pd.read_csv("../../data/data_b_raw/2b302_1229_경기대학교.csv", encoding='cp949')
univ.head()

In [ ]:
univ = univ[['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15']]
univ.dropna(inplace=True)
univ.rename(columns={'Unnamed: 13':'도', 'Unnamed: 14':'시', 'Unnamed: 15':'세부'}, inplace=True) # 세부 : 구, 읍, 면, 로, 길
univ.head()

In [ ]:
print('도 :', univ['도'].unique())
print('시 :', univ['시'].unique())
print('세부 :', univ['세부'].unique())

In [ ]:
# sanity check
city = set(np.where(univ['시'].isin(city_list))[0]) # 시(city_list)에 속해있는 대학교들의 index
district = set(np.where(univ['세부'].isin(district_list_))[0]) # 구(district_list)에 속해있는 대학교들의 index
print(set.intersection(city,district)) # city와 district는 겹치지 않는다.
print(set(np.arange(248)) == set.union(city,district)) # city와 district를 합치면 248개가 다 나온다.

In [ ]:
# 지역 컬럼 생성 : 구가 없는 시면 시 이름을, 구가 있는 시면 구 이름을 배정한다.
city_series = univ['시'][univ['시'].isin(city_list)]
district_series = univ['세부'][univ['세부'].isin(district_list_)]
univ['지역'] = pd.concat([city_series, district_series])
univ.head()

In [ ]:
# 앞서 선정한 42개 지역 목록에 포함된다. 즉, 일치하지는 않는다. 왜냐하면, 대학교가 없는 지역이 존재하기 때문이다.
print(sorted(univ['지역'].unique())==regions_shortened)
print(len(univ['지역'].unique()))
print(set(univ['지역'].unique()).issubset(set(regions_shortened)))

In [ ]:
# 지역 컬럼의 이름을 regions에서의 양식과 일치시킨다.
univ['지역'].replace(shorten, inplace=True) 
univ.head()

In [ ]:
# 각 지역별 대학교의 개수 데이터 생성
univ_ = pd.DataFrame(index = regions)
univ_['대학교 수'] = [sum(univ['지역']==region) for region in regions]
univ_.head()

## 아파트 수 : `apartment`

In [ ]:
apartment = pd.read_csv("../../data/data_b_raw/2b101_1229_경기도 아파트 현황.csv")
sorted(apartment['경기도 아파트현황'].unique()) == regions # 앞서 선정한 42개 지역 목록과 일치한다.
# 경기지역 아파트 목록(윤겸님)
# 데이터를 얻은 곳 : https://data.gg.go.kr/portal/data/service/selectServicePage.do?infId=VUPYJVKMYEYIKOQDILSR30099546&infSeq=1

In [ ]:
apartment.rename(columns={'경기도 아파트현황':'지역', '아파트수':'아파트 수'}, inplace=True)
apartment.set_index('지역',inplace=True)
apartment = apartment.reindex(regions)
apartment.head()

## 지하철역 수 : `subway_`

In [ ]:
subway = pd.read_csv("../../data/data_b_raw/2b204_1229_국가철도공단_1_9_경전철.csv", encoding='CP949')
subway.head()
# 경기지역 지하철역 목록(연수님)
# 데이터를 얻은 곳 : https://www.data.go.kr/data/15041113/fileData.do 등 다수

In [ ]:
cmask = subway.isin(regions+regions_shortened).any(axis='rows') # column mask
# subway의 원소들 중 '42개 지역'을 하나라도 포함하고 있는 열들에 대한 mask
cmask = np.where(cmask)[0] # [5,6,7]
subway = subway.iloc[:,cmask] # subway에서 5, 6, 7번에 해당하는 열들만 추린다.
subway.head()

In [ ]:
rmask = subway.isin(regions+regions_shortened).any(axis='columns') # row mask
# subway의 원소들 중 '42개 지역'을 하나라도 포함하고 있는 행들에 대한 mask
subway = subway[rmask] # subway에서 '42개 지역'을 하나라도 포함하고 있는 행들만 추린다.
subway.rename(columns={'Unnamed: 5':'도', 'Unnamed: 6':'시', 'Unnamed: 7':'세부'}, inplace=True) # 컬럼 명을 '도'와 '시', '세부'로 변경
subway.head()

In [ ]:
# sanity check
# 244 = 2+166+76 = 2+(144+22)+76, 잘 됐다.
print(subway.isin(regions+regions_shortened)["도"].sum()) # 2
print(subway.isin(regions+regions_shortened)["시"].sum()) # 166
print(subway.isin(regions+regions_shortened)["세부"].sum()) # 76
print(subway.isin(city_list)["도"].sum()) # 2
print(subway.isin(city_list)["시"].sum()) # 144
print(subway.isin(district_list)["시"].sum()) # 22
print(subway.isin(district_list)["세부"].sum()) # 76

In [ ]:
city1 = set(np.where(subway['도'].isin(city_list))[0])
city2 = set(np.where(subway['시'].isin(city_list))[0])
city = set.union(city1,city2) # '시'에 속해있는 지하철역들의 index
district1 = set(np.where(subway['시'].isin(district_list))[0])
district2 = set(np.where(subway['세부'].isin(district_list))[0])
district = set.union(district1, district2) # '구'에 속해있는 지하철역들의 index
print(set.intersection(city,district)) # city와 district는 겹치지 않는다.
print(set(np.arange(244)) == set.union(city,district)) # city와 district를 합치면 244개가 다 나온다.

In [ ]:
# 지역 컬럼 생성 : 구가 없는 시면 시 이름을, 구가 있는 시면 구 이름을 배정한다.
city_series1 = subway['도'][subway['도'].isin(city_list)]
city_series2 = subway['시'][subway['시'].isin(city_list)]
district_series1 = subway['시'][subway['시'].isin(district_list_)]
district_series2 = subway['세부'][subway['세부'].isin(district_list_)]
subway['지역'] = pd.concat([city_series1, city_series2, district_series1, district_series2])
subway

In [ ]:
# 앞서 선정한 42개 지역 목록에 포함된다. 즉, 일치하지는 않는다. 왜냐하면, 대학교가 없는 지역이 존재하기 때문이다.
print(sorted(subway['지역'].unique())==regions_shortened)
print(len(subway['지역'].unique()))
print(set(subway['지역'].unique()).issubset(set(regions_shortened)))

In [ ]:
# 지역 컬럼의 이름을 regions에서의 양식과 일치시킨다.
subway['지역'].replace(lengthen, inplace=True) 
subway.head()

In [ ]:
# 각 지역별 지하철역 개수 데이터 생성
subway_ = pd.DataFrame(index = regions)
subway_['지하철역 수'] = [sum(subway['지역']==region) for region in regions]
subway_.head()

## 버스정류장 수 : `busstop`

In [ ]:
busstop = pd.read_csv("../../data/data_b_raw/2b401_0104_Bus_Stop_final.csv", encoding='CP949')
busstop.rename(columns={'Unnamed: 0':'지역','정류장수':'정류장 수'}, inplace=True)
busstop.head()
# 경기지역 버스정류장 목록(현우님)
# 데이터를 얻은 곳 : [출처필요]

In [ ]:
regions_temp = list(busstop['지역'])
# regions == [region.replace('_', ' ') for region in regions_temp] # True
busstop['지역'].replace(dict(zip(regions_temp, regions)), inplace=True)
busstop.set_index('지역', inplace=True)
busstop.head()

## 환승역 수 : transfer

In [ ]:
transfer = pd.read_csv("../../data/data_b_raw/2b501_0104_환승역.csv", encoding='CP949')
transfer.set_index('지역', inplace=True)
transfer.head()

## 행복주택 : `happyhouse_`

In [ ]:
happyhouse = pd.read_csv("../../data/data_b_raw/2b206_0111_경기도 행복주택주거지현황.csv")
# set(happy['지역']).issubset(set(regions)) # True
# happy['지역'].nunique() # 25
happyhouse.head()

In [ ]:
happyhouse_ = pd.DataFrame(index=regions)
happyhouse_['행복주택 수'] = happyhouse['지역'].value_counts()
happyhouse_ = happyhouse_.fillna(0)
happyhouse_['행복주택 수'] = happyhouse_['행복주택 수'].astype(int)
happyhouse_.head()

## 20-30대 인구, 대학생 수

In [ ]:
undergraduate = pd.read_csv("../../data/data_b_raw/2b106_0120_대학생수.csv", index_col='지역')
undergraduate.columns = ['대학생 수']
undergraduate.head()

In [ ]:
pop2030 = pd.read_csv("../../data/data_b_raw/2b105_0120_2030인구.csv", encoding='cp949', index_col='지역')
pop2030.head()

In [ ]:
pop2030 = pop2030[['구별 인구']]
pop2030.columns = ['2030인구']
pop2030['2030인구'].astype(int)
pop2030.head()

## 면적

In [ ]:
areas = pd.read_excel("../../data/data_b_raw/2b512_0131_areas.xls")
areas.columns = ['지역', '면적']
areas['면적'] = areas['면적'].apply(lambda Str:Str[:len(Str)-1])
areas['면적'] = areas['면적'].apply(lambda x:float(x))
areas.set_index('지역', inplace=True)
areas.head()

# feature 데이터를 하나로 통합

In [ ]:
collected_data = pd.concat([locations, zcount, land_use_, pop, univ_, apartment, subway_, busstop, transfer, happyhouse_,
                            undergraduate, pop2030, areas], axis = 1)
collected_data.index.name = '지역'
collected_data.head()

In [ ]:

collected_data = collected_data.reindex(columns = ['쏘카존 수', '인구', '2030인구', '아파트 수', '행복주택 수', '지하철역 수', '환승역 수', '정류장 수', '대학교 수', '대학생 수',
                                                   '녹지', '주거', '공업', '상업', '위도', '경도', '면적'])
collected_data

In [ ]:
collected_data.shape

In [ ]:
collected_data.to_csv("../../data/data_c_final/2c201_1230_collected_data.csv", encoding="utf-8-sig")
savedf(collected_data, "../../data/data_c_final/2c202_1230_collected_data.png")
collected_data.to_csv("results/1b201_1230_collected_data.csv", encoding="utf-8-sig")
savedf(collected_data, "results/1b202_1230_collected_data.png")